In [ ]:
import math
import numpy as np
import random
from time import time

import pandas as pd
from downcast import reduce

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb

import optuna
import joblib

from model import Autoencoder, Regressor
import torch
from torch import nn, optim
from utils import *

In [ ]:
import pprint
pp = pprint.PrettyPrinter()

# PREPARE FEATURE SETS

In [ ]:
IOWA_PATH = '../../datasets/train_data_iowa.csv'
CF_PATH = "../../datasets/crafted_features.csv"

In [ ]:
features = [
    "location",
    "location_x", "location_y",
    "restaurant_location_x", "restaurant_location_y", 
    "order_time",
    "etd",
    "restaurant_queue",
    "max_pre_shift",
    "max_post_shift",
    "restaurants_before_customer",
    "customers_before_customer",
    "len_vehicle_route_to_customer",
]

for i in range(24):
    features.append(f"vehicle_route_to_customer_pos_x_{i}")
    features.append(f"vehicle_route_to_customer_pos_y_{i}")
    features.append(f"vehicle_route_to_customer_action_{i}")
    features.append(f"vehicle_route_to_customer_time_action_{i}") 

In [ ]:
# Import data
start_time = time()

usecols=[*features, "atd"]

meta = pd.read_csv(IOWA_PATH, header=0, sep=";", usecols = usecols, nrows=2)
meta = reduce(meta)
dtypes = dict(meta.dtypes)

raw = pd.read_csv(IOWA_PATH, header=0, sep=";", usecols = usecols, dtype=dtypes) 

print(f"Elapsed time: {time() - start_time} seconds")
print(raw.info(verbose=False, memory_usage="deep"))
X_raw = raw.loc[:, raw.columns != 'atd']
y_raw = raw['atd'] - raw['etd']

pd.set_option("display.max_columns", len(meta.columns))
raw

# Feature Engineering

Features used in Hildebrandt et al. (2020):
<ul>
    <li>n_stops: sum(vehicle_route_to_customer_action_i = 1 or 2)</li>
    <li>n_pickup_stops: sum(vehicle_route_to_customer_action_i = 1)</li>
    <li>n_delivery_stops: sum(vehicle_route_to_customer_action_i = 2)</li>
    <li>max_pre_shift: already given</li>    
    <li>max_post_shift: already given</li>
    <li>prep_time: already given ( == restaurant_queue)</li>
    <li>order_time: already given</li>
    <li>eta_pom: already given</li>
    <li>customer_location: already given</li>
    <li>restaurant_location: already given</li>
</ul>

In [ ]:
#Define strings to identify needed columns for each feature we want to craft
query_strings = {
    "n_stops" : ["vehicle_route_to_customer_action"],
        
    "n_pickup_stops" : ["vehicle_route_to_customer_action"],
    
    "n_delivery_stops" : ["vehicle_route_to_customer_action"],
}

raw_feats = [
    "location_x", "location_y",
    "restaurant_location_x", "restaurant_location_y",
    "etd", 
    "atd", 
    "order_time", 
    "max_pre_shift", 
    "max_post_shift", 
    "restaurant_queue",
    "restaurants_before_customer", 
    "customers_before_customer",
    "len_vehicle_route_to_customer",
    
    
]

mask = pd.DataFrame()
feats = pd.DataFrame()

# First, add used raw features to feats
for feat in raw_feats:
    feats[feat] = raw[feat]

# Craft features and add to feats
for key,value in query_strings.items():
    
    needed_columns = [col for col in raw.columns if any(x in col for x in value)]
    inp = raw[needed_columns]
    
    if key == "n_stops":
        for col in inp:
            mask[col] = (inp[col] > 0) & (inp[col] < 3)
            feats[key] = mask.sum(axis=1)
    
    if key == "n_pickup_stops": 
        for col in inp:
            mask[col] = inp[col] == 1
            feats[key] = mask.sum(axis=1)
    
    if key == "n_delivery_stops": 
        for col in inp:
            mask[col] = inp[col] == 2
            feats[key] = mask.sum(axis=1)
    
    if key == "dropoff_time" :
        inp_copy = inp.drop(["restaurant_location_x", "restaurant_location_y"], axis=1)
        actions = inp_copy[[f for f in inp_copy if "vehicle_route_to_customer_action" in f]] 
        
        dropoff_times = []
        
        for index, row in actions.head(50).iterrows():
            dropoff_action = []
            for i, v in row.items():
                if v == 4:
                    dropoff_action.append(i)
            customer_dropoff = dropoff_action[-1]
            print(index)
            dropoff_times.append(
                inp_copy.at[index, f"vehicle_route_to_customer_time_action_{customer_dropoff[-1]}"]
            )
        dropoff_np = np.asarray(dropoff_times)

In [ ]:
feats.to_csv(CF_PATH, sep=";")

In [ ]:
crafted = pd.read_csv(CF_PATH, sep=";", index_col=[0])

X_crafted = crafted.loc[:, crafted.columns != 'atd']
y_crafted = crafted['atd'] - crafted['etd']

# Data description

## Temporal distributions

In [ ]:
def displot(data, xlabel, ylabel, filepath=None, kind="kde", bw_adjust=2):
    ax = sns.displot(data, 
            kind=kind,
            bw_adjust=bw_adjust,
            height=4, aspect=6/4,
            legend=True)
    ax.set(xlabel=xlabel, ylabel=ylabel)
    if filepath != None:
        ax.savefig(filepath)
    plt.show(ax)

In [ ]:
displot(
    data = crafted["order_time"], 
    xlabel = "Order Time (in min)", 
    ylabel = "Frequency (relative)", 
    filepath = "Plots/order_time_dist.png"
)

In [ ]:
displot(
    data = crafted["atd"]-crafted["etd"],
    xlabel = "Delivery delay (in min)",
    ylabel = "Frequency (relative)",
    filepath = "Plots/delivery_delay.png",
)

In [ ]:
displot(
    data = crafted["restaurant_queue"],
    xlabel = "Preparation time (in min)",
    ylabel = "Frequency (relative)",
    filepath = "Plots/prep_time.png",
)

## Spatial distributions

In [ ]:
sns.displot(
    crafted["restaurant_queue"],
    kind="kde",
    bw_adjust = 3,
    height = 4, aspect=6/4
).savefig("Plots/prep_time.png")

Idea: Scatterplot x Heatmap?

In [ ]:
customer_locations = np.asarray(list(set(zip(raw.location_x, raw.location_y))))
customer_locations_x = [t[0] for t in customer_locations]
customer_locations_y = [t[1] for t in customer_locations]

restaurant_locations = list(set(zip(raw.restaurant_location_x, raw.restaurant_location_y)))
restaurant_locations_x = [t[0] for t in restaurant_locations]
restaurant_locations_y = [t[1] for t in restaurant_locations]

print(customer_locations.shape)
plt.scatter(customer_locations_x, customer_locations_y, s=0.1)
plt.scatter(restaurant_locations_x, restaurant_locations_y, s=10, marker="h")
plt.savefig("Plots/spatial_dist.png")

plt.show()

# train() for NN, temporary in this notebook

# Study

### Helper functions ###

In [ ]:
def plot_convergence(sample_sizes, results, title):
    plt.xlabel("Sample size")
    plt.ylabel("Mean squared error")
    plt.plot(sample_sizes, results)
    plt.savefig(f"Plots/{title}.png")

def best_iteration(evals_result):
    best = evals_result[0]
    for i in evals_result:
        if best > i:
            best = i
    return best 

### Training functions

In [ ]:
def ensemble_train(X_train, y_train, X_test, y_test, params):
    
    train_set = lgb.Dataset(X_train,y_train)
    val_set = lgb.Dataset(X_test, y_test, reference=train_set)
    
    evals_result = {}
    bst = lgb.train(
        params,
        train_set=train_set,
        valid_sets=[val_set, train_set],
        verbose_eval=5,
        evals_result = evals_result,
    )
    best_mse = best_iteration(evals_result["valid_0"]["l2"])
    print(best_mse)
    return bst

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_crafted,y_crafted, train_size=0.8, random_state=42)

params = {
    "boosting_type" : "gbdt",
    "metrics" : "l2",
    "learning_rate" : 0.02, 
    "num_threads"  : 6,
    "random_state" : 42,
    "force_row_wise" : True,
    "n_estimators" : 10,
    "early_stopping_rounds" : 20,
}

bst = ensemble_train(X_train, y_train, X_test, y_test, params)

In [ ]:
def nn_train(model, data, feature_list, params):
    
    # Set the seed for reproducability
    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("Importing data.")

    etd_dataset = ETDData(data=data, feature_list=feature_list, objective=model.name)
    split = DataSplit(etd_dataset, shuffle=True)
    trainloader, _, testloader = split.get_split(batch_size=params["batch_size"], num_workers=8)
    
    print("Start training.")
    patience = params["patience"]
    criterion = params["criterion"]  
    optimizer = params["optimizer"]

    
    early_stopping = EarlyStopping(patience=params["patience"], verbose=True) 
    epochs = params["epochs"]
    
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.float().to(device)
            labels = labels.float().view(-1, model.view).to(device) 
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels) 
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
        test_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs = inputs.float().to(device)
                labels = labels.float().view(-1, model.view).to(device) 
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)
                test_loss += batch_loss.item()
        print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss / len(trainloader):.3f}.. "
                f"Test loss: {test_loss / len(testloader):.3f}.. ")
        early_stopping(test_loss / len(testloader), model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        model.train()
        
    print('Finished Training')
    model.load_state_dict(torch.load('checkpoint.pt'))
    model.save(model, 'perceptron.pth')
    return model, abs(early_stopping.best_score)

## Part 1:  Sample size testing

With the first part, we seek to examine the convergence behavior of our models and answer following question: How many samples are enough to train the model without ? 
We determine the answer to that question graphically. For that, we construct plots where the x-axis represents the number of samples used in the corresponding training instance, and the y-axis represents the corresponding L<sub>2</sub>-loss measured with the mean squared error.

### Test 1.1: Tree-based ensembles: GBDT and RF (LightGBM Implementation)

In [ ]:
### Convergence Test for LightGBM's GBDT ###
def sample_size_ensembles(X, y, params, title, start=1000, stop=100000, step=1000):
    
    sample_sizes = np.arange(start=start, stop=stop, step=step)
    results = []
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)
    
    for rows in sample_sizes:
        
        train_set = lgb.Dataset(X_train[:rows],y_train[:rows])
        val_set = lgb.Dataset(X_test, y_test, reference=train_set)
        evals_result = {}
        bst = lgb.train(
            params,
            train_set=train_set,
            valid_sets=[val_set, train_set],
            valid_names=["Validation error", "Train error"],
            verbose_eval=5,
            evals_result = evals_result,
        )
        best_mse = best_iteration(evals_result["Validation error"]["l2"])
        results.append(best_mse)
    plot_convergence(sample_sizes, results, title)

In [ ]:
params_gbdt = {
    "boosting_type" : "gbdt",
    "metrics" : "l2",
    "learning_rate" : 0.02, 
    "num_threads"  : 6,
    "random_state" : 42,
    "force_row_wise" : True,
    "n_estimators" : 1000,
    "early_stopping_rounds" : 20,
}

params_rf = {
    "boosting_type" : "rf",
    "learning_rate" : 0.02,
    "metrics" : "l2",
    "n_estimators" : 1000,
    "bagging_fraction" : 0.632,
    "bagging_freq" : 1,
    "feature_fraction" : 0.632,
    "num_threads"  : 6,
    "random_state" : 42,
    "force_row_wise" : True,
    "early_stopping" : 20,
}

In [ ]:
sample_size_ensembles(X_raw, y_raw, params_rf, "RF_SampleSizeTest_Raw")

In [ ]:
sample_size_ensembles(X_raw, y_raw, params_gbdt, "GBDT_SampleSizeTest_Raw")

In [ ]:
sample_size_ensembles(X_crafted, y_crafted, params_gbdt, "GBDT_SampleSizeTest_Crafted")

In [ ]:
sample_size_ensembles(X_crafted, y_crafted, params_rf, "RF_SampleSizeTest_Crafted")

### Test 1.2: Linear Regression

In [ ]:
### Convergence test for Scikit-Learn's Linear Regression ###
def sample_size_lr(X, y, title, start=1000, stop=100000, step=1000):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)
    sample_sizes = np.arange(start=start, stop=stop, step=step)
    results = []
    
    for rows in sample_sizes:
        lr = LinearRegression()
        lr.fit(X_train[:rows], y_train[:rows])
        mse = mean_squared_error(y_test, lr.predict(X_test))
        print(f"Sample size - error : {rows} - {mse}")
        results.append(mse)
    plot_convergence(sample_sizes, results, title)

In [ ]:
sample_size_lr(X_raw, y_raw, "LR_SampleSize_Raw")

In [ ]:
sample_size_lr(X_crafted, y_crafted, "LR_SampleSize_Crafted")

### Test 1.3: Single Layer Perceptron

In [ ]:
sample_sizes = [1000,10000,100000]
results = []

n_features = len(features)
n_hidden = math.ceil(n_features * (1 / 2))
n_code = math.ceil(n_hidden * (1 / 2))

ae = Autoencoder(n_features=n_features, n_hidden=n_hidden, n_code=n_code)
slp = Regressor(n_features = n_features, n_hidden = n_hidden, n_output = 1)

params = {
        "patience" : 10,
        "criterion" : nn.MSELoss(),
        "optimizer" : optim.Adam(slp.parameters(), lr=0.0001),
        "epochs" : 100,
        "batch_size" : 50,
}

for rows in sample_sizes:
    print(f"Sample size {rows}")
    model, mse = train(slp, raw, features, params)
    results.append(mse)
    print(f"Mean squared error: {mse} for sample size: {rows}")
plot_convergence(sample_sizes, results, "nn_sample_size")

## Part 2: Hyperparameter optimization (HPO)

### Helper functions 

In [ ]:
def print_results(study):
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trial
    print("Value: {}".format(trial.value))
    print("Params: ")
    for key, value in trial.params.items():
        print("{}: {}".format(key, value))   

### HPO Objective functions for Ensembles

In [ ]:
def hpo_trees(trial, X, y, mode):
    
    sample_size=100000
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
    train_set = lgb.Dataset(X_train[:sample_size],y_train[:sample_size])
    valid_set = lgb.Dataset(X_test, y_test)
    
    params = {
        "gbdt" : {
            "boosting_type" : "gbdt",
            "metric" : "l2",
            "objective" : "regression",
            "learning_rate" : trial.suggest_uniform("learning_rate", 0.01, 0.05),
            "max_depth" : trial.suggest_int("max_depth", 20, 80),
            "feature_fraction" : trial.suggest_uniform("feature_fraction", 0.1, 1.0),
            "num_leaves" : trial.suggest_int("num_leaves", 20, 300),
            "min_data_in_leaf" : trial.suggest_int("min_data_in_leaf", 1, 40),
            "max_bin" : 1000,
            "feature_pre_filter" : False,
            "num_threads"  : 6,
            "random_state" : 42,
            "force_row_wise" : True, 
            "num_boost_round": 1000,
            "early_stopping" : 50,
        },
        "rf" : {
            "boosting_type" : "rf",
            "metric" : "l2", 
            "objective" : "regression",
            "n_estimators" : trial.suggest_int("n_estimators", 500, 1000),
            "learning_rate" : trial.suggest_uniform("learning_rate", 0.01, 0.05),
            "max_depth" : trial.suggest_int("max_depth", 20,60),
            "feature_fraction" : trial.suggest_uniform("feature_fraction", 0.50, 0.99),
            "bagging_fraction" : trial.suggest_uniform("bagging_fraction", 0.50, 0.99), 
            "bagging_freq" : trial.suggest_int("bagging_frequency", 1, 20),
            "num_leaves" : trial.suggest_int("num_leaves", 20, 300),
            "min_data_in_leaf" : trial.suggest_int("min_child_samples", 1, 40),
            "feature_pre_filter" : False,
            "max_bin" : 1000,
            "num_threads"  : 6,
            "random_state" : 42,
            "force_row_wise" : True, 
            "num_boost_round": 1000,
            "early_stopping" : 50,
        }      
    }
    evals_result = {}
    bst = lgb.train(
        params[mode],
        train_set=train_set,
        valid_sets=[valid_set, train_set],
        valid_names=["Validation error", "Train error"],
        verbose_eval=0,
        evals_result = evals_result
    )
    best_mse = best_iteration(evals_result["Validation error"]["l2"])
    print(f"Best iteration: {best_mse}")
    return best_mse

In [ ]:
study = optuna.create_study(
    direction="minimize", 
    sampler=optuna.samplers.CmaEsSampler(seed=42),
    pruner=optuna.pruners.SuccessiveHalvingPruner()
)
start_time = time()
study.optimize(lambda trial: hpo_trees(trial, X_raw, y_raw, "gbdt"), n_trials=100)
print(f"Elapsed time: {time() - start_time} seconds")

joblib.dump(study, "gbdt_raw.pkl")

print_results(study)

In [ ]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.CmaEsSampler(seed=42))
study.optimize(lambda trial: hpo_trees(trial, X_raw, y_raw, "rf"), n_trials=500)

joblib.dump(study, "rf_raw.pkl")

print_results(study)

In [ ]:
study = joblib.load("gbdt_raw.pkl")
print_results(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

# Part 3: Introducing Noise